### 라이브러리 임포트

In [3]:
import tkinter as tk
from PIL import Image, ImageTk

### 조 이름, 조원 정보 입력 기능

- 주의 사항
  - 팀 정보 저장 형태 : `[{'team_number': 1, 'team_member': '다람쥐 고양이 레고 '}, {'team_number': 2, 'team_member': '최현정 전혜준 김경환 '}]`

- 개선할 점
  - 유효성 검사 기능 추후 검토
  - 조 번호 표시되어 있는 이미지로 변경 필요
  - 커서 효과 개선
  - 사용자 입력시 왼쪽상단에 작은 박스로 입력 문자가 표시됨 왤까?

In [4]:
import tkinter as tk 
import os

# 팀 정보 저장 리스트
team_info = []  # 팀 정보를 저장할 리스트
input_texts = ["" for _ in range(7)]  # 각 입력칸의 텍스트를 저장할 리스트
max_teams = 7  # 최대 7개의 팀
img_objects = []  # 이미지 참조를 유지하기 위한 리스트 (이미지 해제를 방지)
image_id = None  # 현재 캔버스에 표시된 이미지 객체의 ID 저장
image_ids = []  # info 페이지에서 추가된 이미지를 저장할 리스트
input_positions = [100]  # 첫 번째 입력칸은 이미 디자인되어 있으므로 위치 저장
text_ids = [None for _ in range(7)]  # 각 입력칸의 텍스트 ID를 저장할 리스트
cursor_id = None  # 현재 활성화된 커서의 ID
edit_button_ids = [None for _ in range(7)]  # 각 입력칸의 수정하기 버튼 ID를 저장할 리스트
edit_mode = [False for _ in range(7)]  # 각 입력칸의 수정 모드 상태를 저장할 리스트

# 입력칸 간격 설정
input_spacing = 75  # 입력칸 간의 간격

# 새로운 전역 변수 추가
active_input = None  # 현재 활성화된 입력칸의 인덱스

# 텍스트 입력받는 함수
def handle_input(event):
    global active_input
    if active_input is not None:
        if event.keysym == 'BackSpace':
            input_texts[active_input] = input_texts[active_input][:-1]
        elif event.keysym == 'space':
            input_texts[active_input] += ' '
        elif len(event.char) == 1:
            input_texts[active_input] += event.char
        update_text_display(active_input)
        update_cursor_position(active_input)

# 텍스트 디스플레이 업데이트 함수
def update_text_display(index):
    if text_ids[index]:
        canvas.delete(text_ids[index])
    y_position = input_positions[index]
    text_ids[index] = canvas.create_text(180, y_position + 25, text=input_texts[index], anchor='w', font=('Arial', 12))

# 커서 위치 업데이트 함수
def update_cursor_position(index):
    global cursor_id
    if cursor_id:
        canvas.delete(cursor_id)
    y_position = input_positions[index]
    text_width = canvas.bbox(text_ids[index])[2] - canvas.bbox(text_ids[index])[0] if text_ids[index] else 0
    cursor_id = canvas.create_line(180 + text_width, y_position + 10, 180 + text_width, y_position + 40, fill='black')
    blink_cursor()

# 마우스 클릭 시 입력칸, 버튼 처리
def handle_page(event):
    global active_input, cursor_id

    if cursor_id:
        canvas.delete(cursor_id)
        cursor_id = None

    for i, y_position in enumerate(input_positions):
        if 160 < event.x < 1000 and y_position < event.y < y_position + 50:
            print(f"입력칸 {i+1} 클릭됨")
            activate_input(i)
            return
    
    # 추가 버튼 클릭 좌표 확인 (이전과 동일)
    for i, y_position in enumerate(input_positions):
        if 1150 < event.x < 1250 and y_position < event.y < y_position + 50:
            if i >= len(team_info):  # 새로운 팀 정보 추가
                handle_add_button(i)
            else:  # 이미 존재하는 팀 정보 수정
                handle_edit_button(i)
            return

    # 수정하기 버튼 클릭 좌표 확인
    for i in range(len(team_info)):
        button_x, button_y = get_button_position(i)
        if button_x - 40 < event.x < button_x + 40 and button_y - 20 < event.y < button_y + 20:
            handle_edit_button(i)
            return

    active_input = None  # 입력칸 외 영역 클릭 시 활성 입력칸 초기화

# 입력칸 활성화 함수
def activate_input(index):
    global active_input
    active_input = index
    canvas.focus_set()
    update_cursor_position(index)

# 추가 버튼 처리 함수
def handle_add_button(index):
    print(f"추가 버튼 클릭됨 (입력칸 {index+1})")
    if input_texts[index]:  # 입력된 내용이 있는 경우에만 저장
        team_number = len(team_info) + 1
        team_member = input_texts[index]
        team_info.append({'team_number': team_number, 'team_member': team_member})
        print(f"저장된 팀 정보: {team_info}")
        if len(input_positions) < max_teams:
            add_input_field()
        display_edit_button(index)

# 수정하기 버튼 처리 함수
def handle_edit_button(index):
    print(f"수정하기 버튼 클릭됨 (입력칸 {index+1})")
    if input_texts[index]:  # 입력된 내용이 있는 경우에만 업데이트
        team_info[index]['team_member'] = input_texts[index]
        print(f"팀 정보 업데이트: {team_info}")
    else:
        print("입력된 내용이 없어 업데이트하지 않았습니다.")

# 커서 깜빡임 효과 함수
def blink_cursor():
    global cursor_id
    if cursor_id:
        current_state = canvas.itemcget(cursor_id, 'state')
        new_state = 'hidden' if current_state == 'normal' else 'normal'
        canvas.itemconfigure(cursor_id, state=new_state)
    if active_input is not None:
        canvas.after(600, blink_cursor)  # 깜빡임 간격을 600ms로 설정

# 수정하기 버튼 위치 계산 함수
def get_button_position(index):
    if index == 0:
        # 첫 번째 버튼의 위치 (직접 설정)
        return 1070, 130  # 예시 좌표, 필요에 따라 조정하세요
    else:
        # 두 번째 버튼부터의 위치
        y_position = input_positions[index]
        return 1070, y_position + 25   # x 좌표는 1070으로 고정, y 좌표는 입력칸 중앙

# 새 입력칸 추가 함수
def add_input_field():
    if len(input_positions) < max_teams:
        y_position = input_positions[0] + len(input_positions) * input_spacing
        input_positions.append(y_position)
        
        try:
            img_entry = tk.PhotoImage(file='../icon_img/info.png')
            img_objects.append(img_entry)
            image_id = canvas.create_image(635, y_position + 25, image=img_entry)
            image_ids.append(image_id)
            
            # 새 입력칸에 대한 텍스트 디스플레이 초기화
            new_index = len(input_positions) - 1
            input_texts[new_index] = ""
            update_text_display(new_index)
            
            print(f"새 입력칸 추가됨: {len(input_positions)}번째")
        except tk.TclError as e:
            print(f"이미지 로딩 오류: {e}")
            print(f"시도한 경로: ../icon_img/info.png")

# 수정하기 버튼 표시 함수
def display_edit_button(index):
    try:
        edit_img = tk.PhotoImage(file='../icon_img/edit_button.png')
        img_objects.append(edit_img)
        button_x, button_y = get_button_position(index)
        edit_button_ids[index] = canvas.create_image(button_x, button_y, image=edit_img)
    except tk.TclError as e:
        print(f"이미지 로딩 오류: {e}")
        print(f"시도한 경로: ../icon_img/edit_button.png")

# 메인 윈도우 생성
root = tk.Tk()
root.title("누가 누가 꼴찌할까 - 팀 정보 입력")

# Canvas 생성
canvas = tk.Canvas(root, width=1280, height=720)
canvas.pack()

# 현재 작업 디렉토리 출력
print("현재 작업 디렉토리:", os.getcwd())

# 첫 번째 배경 이미지 로드 (첫 번째 입력칸과 추가 버튼 포함)
try:
    img = tk.PhotoImage(file='../bg_img/info_page.png')
    img_objects.append(img)
    image_id = canvas.create_image(640, 360, image=img)
    image_ids.append(image_id)
except tk.TclError as e:
    print(f"이미지 로딩 오류: {e}")
    print(f"시도한 경로: ../bg_img/info_page.png")
    img = tk.PhotoImage(width=1, height=1)  # 빈 이미지

# 마우스 클릭 이벤트 바인딩
canvas.bind("<Button-1>", handle_page)

# 키보드 입력 이벤트 바인딩
root.bind("<Key>", handle_input)

# 창 실행
root.mainloop()

현재 작업 디렉토리: c:\Users\K2SC2000\Desktop\Who-will-be-last-\game_code
입력칸 1 클릭됨
추가 버튼 클릭됨 (입력칸 1)
저장된 팀 정보: [{'team_number': 1, 'team_member': 'ㅗㅜㅕㅗ'}]
새 입력칸 추가됨: 2번째
입력칸 2 클릭됨
입력칸 2 클릭됨
